#  Submit Molecule Geometry Optimization

In [ ]:
%aiida

In [ ]:
# General imports
import nglview
import numpy as np
import ipywidgets as ipw
from collections import OrderedDict
from ase.data import vdw_radii
from ase import Atoms
from IPython.display import display, clear_output, HTML
import itertools

# AiiDA imports
from aiida_cp2k.utils import Cp2kInput
from aiida.orm import SinglefileData
from aiidalab_widgets_base import CodeDropdown, SubmitButtonWidget,  StructureBrowserWidget
from aiida_cp2k.workchains.base import Cp2kBaseWorkChain

# Custom imports
from apps.surfaces.widgets.metadata import MetadataWidget
from apps.surfaces.widgets import analyze_structure
from apps.surfaces.widgets.cp2k2dict import CP2K2DICT
from apps.surfaces.widgets.create_xyz_input_files import make_geom_file
from apps.surfaces.widgets.dft_details import DFTDetails
from apps.surfaces.widgets.viewer_details import ViewerDetails
from apps.surfaces.widgets.slab_validity import slab_is_valid
from apps.surfaces.widgets.cp2k_input_validity import validate_input #input_is_valid
from apps.surfaces.widgets.suggested_param import suggested_parameters
from apps.surfaces.widgets.get_cp2k_input import Get_CP2K_Input

from aiidalab_widgets_base import CodeDropdown, StructureManagerWidget, StructureBrowserWidget, StructureUploadWidget, SubmitButtonWidget, SmilesWidget

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
## STRUCTURE SELECTOR
structure_selector = StructureManagerWidget(importers=[
    ("AiiDA database", StructureBrowserWidget()),
    ("Import from computer", StructureUploadWidget()),
    ("SMILES",SmilesWidget()),
    ],
    storable=False, node_class='StructureData')
display(structure_selector)

In [ ]:
## GENERAL PARAMETERS
slab_analyzed = None
atoms = None
details_dict={}
workchain='MoleculeOptWorkChain'
details_dict['workchain']=workchain

In [ ]:
## DESCRIPTION OF CALCULATION
process_description = ipw.Text(description='Process description: ',
                               placeholder='Type the name here.',
                               style={'description_width': '120px'},
                               layout={'width': '70%'})

In [ ]:
# Observer structure selection change.
mol_ids_info_out = ipw.Output()

def on_struct_change(c):
    global details_dict, slab_analyzed, atoms
    if not structure_selector.structure:
        return
    atoms = structure_selector.structure
    atoms.pbc = [1, 1, 1]
    slab_analyzed = analyze_structure.analyze(atoms)
    if slab_analyzed['system_type'] != 'Molecule':
        structure_selector.structure = None
        msg='Only molecules are allowed'
    else:
        details_dict={}
        details_dict['elements']=slab_analyzed['all_elements']
       
        workchain='MoleculeOptWorkChain'
        details_dict['workchain']=workchain        
        dft_details_widget.reset(slab_analyzed)
        msg=slab_analyzed['summary']
    
    with mol_ids_info_out:
        clear_output()
        print(msg)
        
structure_selector.observe(on_struct_change, names='structure')
display(mol_ids_info_out)

In [ ]:
## TOGGLE VIEW FIXED ATOMS
def on_fixed_atoms_btn_click(c):
    if dft_details_widget.btn_fixed_pressed:
        structure_selector.viewer.selection = dft_details_widget.fixed_atoms.value
    else:
        structure_selector.viewer.selection = set()

In [ ]:
# Code
computer_code_dropdown = CodeDropdown(input_plugin='cp2k')#, path_to_root="../../")

# DFT
dft_details_widget = DFTDetails(workchain=workchain,structure_details=slab_analyzed)    
dft_details_widget.btn_fixed_atoms.on_click(on_fixed_atoms_btn_click)

## NUMBER NODES
metadata_widget = MetadataWidget()
if computer_code_dropdown.selected_code:
    metadata_widget.num_mpiprocs_per_machine.value = computer_code_dropdown.selected_code.computer.get_default_mpiprocs_per_machine()

def modify_mpiprocs_per_machine(change):
    metadata_widget.num_mpiprocs_per_machine.value = change['new'].computer.get_default_mpiprocs_per_machine()
    
computer_code_dropdown.observe(modify_mpiprocs_per_machine, names='selected_code')

## PLAIN TEXT INPUT
plain_input=ipw.Textarea(value='', disabled=False, layout={'width': '60%'})
plain_input_accordion = ipw.Accordion(selected_index=None)
plain_input_accordion.children=[plain_input]
plain_input_accordion.set_title(0,'plain input')

## VALIDATE AND CREATE INPUT
create_input=ipw.Button(description='create input', layout={'width': '10%'})

## INSERT PLAIN INPUT IN BUILDER
def get_plain_input():
    error_msg,submit_dict=CP2K2DICT(input_lines=plain_input.value)
    if error_msg != "":
        print(error_msg)
    else:
        input_builder = Cp2kBaseWorkChain.get_builder()
        
        # code
        input_builder.cp2k.code = computer_code_dropdown.selected_code
        
        # structure
        input_builder.cp2k.file.input_xyz = make_geom_file(structure_selector.structure_node, 
                                                           Str("mol.xyz"),spin_u=Str(spin_u),spin_d=Str(spin_d))
        
        
        input_builder.cp2k.metadata = metadata_widget.dict
        input_builder.metadata.label = "MoleculeOptWorkChain"
        input_builder.metadata.description = process_description.value

        input_builder.cp2k.metadata['label'] = "molecule_opt"
        input_builder.cp2k.metadata['description'] = process_description.value
        input_builder.cp2k.parameters = Dict(dict=submit_dict)
    return input_builder        

## CREATE PLAIN INPUT
def on_create_input_btn_click(c):
    global spin_u,spin_d
    dft_details_widget.get_widget_values(details_dict)
    spin_u=details_dict['spin_u']
    spin_d=details_dict['spin_d']
   
    ##FIXERS    
   # input_builder.fixers = {
   #     "fixer_001": ('aiida_cp2k.fixers', 'resubmit_unconverged_geometry'),
   #     #"fixer_001": ('apps.surfaces.fixers', 'resubmit_unconverged_geometry'),
   # }

    num_machines = metadata_widget.dict['options']['resources']['num_machines']
    num_mpiprocs_per_machine = metadata_widget.dict['options']['resources']['num_mpiprocs_per_machine']
    ntasks = num_machines * num_mpiprocs_per_machine
    details_dict['mpi_tasks'] = ntasks
    details_dict['walltime'] = metadata_widget.dict['options']['max_wallclock_seconds']        
        
    inp_dict = Get_CP2K_Input(input_dict = details_dict).inp
    inp_plain = Cp2kInput(inp_dict)
    plain_input.value = inp_plain.render()
    
    can_submit,error_msg=validate_input(slab_analyzed,details_dict)
    if can_submit:
        btn_submit.btn_submit.disabled=False
    else:
        btn_submit.btn_submit.disabled=True
        print(error_msg)
    
create_input.on_click(on_create_input_btn_click)
## END CREATE PLAIN INPUT

## DISPLAY
display(ipw.VBox([
    computer_code_dropdown,
    dft_details_widget,
    plain_input_accordion,
    process_description,
    metadata_widget,
    create_input]))

## DISPLAY SUBMIT BUTTON
btn_submit = SubmitButtonWidget(process=Cp2kBaseWorkChain,widgets_values=get_plain_input)
btn_submit.btn_submit.disabled = True
display(btn_submit)